<a href="https://colab.research.google.com/github/7ev3r/Midterm_assignment_v2/blob/main/Train_ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np
import time

In [24]:
df = pd.read_csv('Clean_whiskey_data.csv')
df = df.drop('Unnamed: 0', axis = 1)
df.shape

(37567, 7)

In [25]:
pd.options.display.max_colwidth = 10000

In [26]:
df.head()

,whiskey,rating,url,User_ID,Whiskey_ID,price_euro,price_dollar
0,royal-lochnagar-2008,88,https://www.whiskybase.com/whiskies/whisky/224840/royal-lochnagar-2008,771,407,190.0,207.10
1,laphroaig-1967-rwd,100,https://www.whiskybase.com/whiskies/whisky/24413/laphroaig-1967-rwd,609,248,65431.0,71319.79
2,laphroaig-1967-rwd,99,https://www.whiskybase.com/whiskies/whisky/24413/laphroaig-1967-rwd,636,248,65431.0,71319.79
3,laphroaig-1967-rwd,99,https://www.whiskybase.com/whiskies/whisky/24413/laphroaig-1967-rwd,766,248,65431.0,71319.79
4,laphroaig-1967-rwd,99,https://www.whiskybase.com/whiskies/whisky/24413/laphroaig-1967-rwd,642,248,65431.0,71319.79


In [27]:
df['whiskey'] = df['whiskey'].str.replace('-', ' ').str.title()

In [28]:
rec_df = df.drop(columns = ['url', 'price_euro', 'price_dollar'])
rec_df = rec_df[['User_ID', 'Whiskey_ID', 'rating']]

In [29]:
rec_match = df[['url', 'whiskey', 'Whiskey_ID', 'price_dollar', 'price_euro']].drop_duplicates('Whiskey_ID')

In [30]:
rec_match.head()

,url,whiskey,Whiskey_ID,price_dollar,price_euro
0,https://www.whiskybase.com/whiskies/whisky/224840/royal-lochnagar-2008,Royal Lochnagar 2008,407,207.10,190.0
1,https://www.whiskybase.com/whiskies/whisky/24413/laphroaig-1967-rwd,Laphroaig 1967 Rwd,248,71319.79,65431.0
38,https://www.whiskybase.com/whiskies/whisky/23249/laphroaig-1970-rwd,Laphroaig 1970 Rwd,250,9808.91,8999.0
55,https://www.whiskybase.com/whiskies/whisky/12078/strathisla-1967-rwd,Strathisla 1967 Rwd,452,32427.50,29750.0
77,https://www.whiskybase.com/whiskies/whisky/8854/bowmore-1966-sa,Bowmore 1966 Sa,78,81409.92,74688.0


In [31]:
rec_df.User_ID.max()

1008

Fitting model with data

In [32]:
from surprise import Reader, Dataset
reader = Reader(rating_scale = (1,100))
data = Dataset.load_from_df(rec_df,reader)

In [33]:
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=0.25, random_state = 100)

In [34]:
svd = SVD()
svd.fit(trainset)
preds = svd.test(testset)

rating the model

In [51]:
accuracy.rmse(preds)

RMSE: 1.8158


1.8158159318032823

Testing the model

In [52]:
def BoozePal(uid, iid, actual):
    pred = svd.predict(uid, iid, actual, verbose=True)
    match = rec_match.loc[rec_match['Whiskey_ID'] == iid]
    return match

In [53]:
BoozePal(852,0, 88)

user: 852        item: 0          r_ui = 88.00   est = 93.47   {'was_impossible': False}


,url,whiskey,Whiskey_ID,price_dollar,price_euro
15132,https://www.whiskybase.com/whiskies/whisky/215080/a-secret-speyside-distillery-1968-br,A Secret Speyside Distillery 1968 Br,0,1392.76,1277.76


Helper function to return recommendations to the user. Used in the whiskey_rater function below

In [57]:
def get_recommendations(user_ratings, num_recs, priceEU):
    new_ratings_df = rec_match.append(user_ratings,ignore_index=True)
    new_ratings_df = new_ratings_df[new_ratings_df['price_euro'] < priceEU]
    new_ratings_df = new_ratings_df.drop(columns=['url', 'whiskey', 'price_dollar', 'price_euro'])

    new_data = Dataset.load_from_df(new_ratings_df,reader)
    svd_new = SVD()
    svd_new.fit(new_data.build_full_trainset())

    list_of_whiskies = []
    for w_id in new_ratings_df['Whiskey_ID'].unique():
        list_of_whiskies.append((w_id, svd_new.predict(new_ratings_df['User_ID'].max(),w_id)[3]))

    ranked_whiskies = sorted(list_of_whiskies, key=lambda x:x[1],reverse=True)

    rec_num = 1
    for i in ranked_whiskies[:num_recs]:
        recommended = rec_match[rec_match['Whiskey_ID'] == i[0]]
        print('Recommendation number:', rec_num)
        print('Whiskey: ' + recommended.values[0][1])
        print('Price: ' + str(recommended.values[0][-1]))
        print('URL: ' + recommended.values[0][0])
        print('\n')
        rec_num +=1



In [58]:
def BoozePal(df, num):
    userID = rec_df.User_ID.max()+1
    num_recs = input('How many recommendations would you like? Please enter a number from 1 to 10:\n')
    
    while int(num_recs) > 10:
        num_recs = input('You entered a number over 10. Please enter a number from 1 to 10 to continue. \n')
    priceUSD = input('Please enter your budget for a bottle of whiskey. $')
    rating_list = []
    
    while num > 0:
        whiskey = df[df['price_euro'] < int(priceEU)].sample(1)
        print('\nPlease rate the following {} whiskies. \n'.format(num))
        print('Whiskey: ' + whiskey.values[0][2])
        print('Price: ' + str(whiskey.values[0][-1]))
        print('URL: ' + whiskey.values[0][1])
        rating = input('How do you rate this whiskey on a scale of 1-100, press n if you are not familiar with it. :\n')
            
        if rating == 'n':
            continue
        
        if int(rating) > 100:
            print('Rating must be below 100!')
            continue
        
        else:
            rating_one_whiskey = {'User_ID':userID,'Whiskey_ID': whiskey['Whiskey_ID'].values[0],'rating': rating}
            rating_list.append(rating_one_whiskey) 
            num -= 1
        time.sleep(.5)
    print('\n'+'-----Making Recommendations-----'+'\n')
    
    time.sleep(1.5)
    get_recommendations(rating_list, int(num_recs), int(priceUSD))

In [59]:
user_ratings= BoozePal(df, 5)

How many recommendations would you like? Please enter a number from 1 to 10:
5
Please enter your budget for a bottle of whiskey. $120


NameError: ignored